# NN - Churn Modelling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

In [2]:
# import data
dataset = pd.read_csv("Churn_Modelling.csv")
dataset.shape

(10000, 14)

In [3]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.iloc[:, 3:13].values
y= dataset.iloc[:,13].values

In [5]:
# categorical variables

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:,1] = labelencoder_X_1.fit_transform(X[:,1])

labelencoder_X_2 = LabelEncoder()
labelencoder_X_1 = LabelEncoder()
X[:,2] = labelencoder_X_2.fit_transform(X[:,2])

onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()

# remove the reference column
X = X[: , 1:]

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=0)

In [7]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [9]:
# initializing the ANN
classifier = Sequential()

# adding the input layer and the first hidden layer
classifier.add(Dense(6 ,   activation='relu',  kernel_initializer='random_uniform', input_shape = (X_train.shape[1],)))
classifier.add(Dense(6 ,   activation='relu',  kernel_initializer='random_uniform'))

# add the output layer
classifier.add(Dense(1 ,   activation='sigmoid',  kernel_initializer='random_uniform'))

# compile the ANN
classifier.compile(optimizer = 'adam',  loss ='binary_crossentropy', metrics = ['accuracy'] )

In [10]:
classifier.fit(X_train, y_train, batch_size = 10, epochs =5)

Epoch 1/5
8000/8000 [==============================] - 2s 201us/step - loss: 0.4812 - acc: 0.7960
Epoch 2/5
8000/8000 [==============================] - 2s 203us/step - loss: 0.4255 - acc: 0.7960
Epoch 3/5
8000/8000 [==============================] - 1s 175us/step - loss: 0.4200 - acc: 0.8091
Epoch 4/5
8000/8000 [==============================] - 1s 169us/step - loss: 0.4172 - acc: 0.8259
Epoch 5/5
8000/8000 [==============================] - 1s 161us/step - loss: 0.4156 - acc: 0.8286


In [11]:
y_pred = classifier.predict_classes(X_test)


# y_pred = classifier.predict(X_test)
# y_pred = (y_pred>0.5)

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

In [12]:
cm

array([[1576,   19],
       [ 310,   95]])

In [13]:
# accuracy
sum([p[0] for p in y_pred]==y_test)/len(y_test)

0.83550000000000002

In [14]:
# evaluate the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [22]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(6 ,   activation='relu',  kernel_initializer='random_uniform', input_shape = (X_train.shape[1],)))
    classifier.add(Dense(6 ,   activation='relu',  kernel_initializer='random_uniform'))
    classifier.add(Dense(1 ,   activation='sigmoid',  kernel_initializer='random_uniform'))
    classifier.compile(optimizer = 'adam',  loss ='binary_crossentropy', metrics = ['accuracy'] )
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs =7)

accuracies = cross_val_score(estimator = classifier, X=X_train, y= y_train, cv=5)

Epoch 1/7
6400/6400 [==============================] - 2s 281us/step - loss: 0.4950 - acc: 0.7980 0s - loss: 0.5122 - acc: 0
Epoch 2/7
6400/6400 [==============================] - 1s 184us/step - loss: 0.4287 - acc: 0.7980
Epoch 3/7
6400/6400 [==============================] - 1s 178us/step - loss: 0.4229 - acc: 0.7980
Epoch 4/7
6400/6400 [==============================] - 1s 189us/step - loss: 0.4187 - acc: 0.8062
Epoch 5/7
6400/6400 [==============================] - 1s 181us/step - loss: 0.4165 - acc: 0.8234
Epoch 6/7
6400/6400 [==============================] - 1s 182us/step - loss: 0.4151 - acc: 0.8277
Epoch 7/7
1600/1600 [==============================] - 1s 336us/step
Epoch 1/7
6400/6400 [==============================] - 2s 347us/step - loss: 0.4942 - acc: 0.7967
Epoch 2/7
6400/6400 [==============================] - 4s 669us/step - loss: 0.4286 - acc: 0.7972
Epoch 3/7
6400/6400 [==============================] - 8s 1ms/step - loss: 0.4234 - acc: 0.7972
Epoch 4/7
6400/6400 [===

In [23]:
print(accuracies.mean())
print(accuracies.std())

0.827374995463
0.00593453806137


In [24]:
# improve the ANN
from keras.layers import Dropout


# initializing the ANN
classifier = Sequential()

# adding the input layer and the first hidden layer
classifier.add(Dense(6 ,   activation='relu',  kernel_initializer='random_uniform', input_shape = (X_train.shape[1],)))
classifier.add(Dropout(rate=0.1))
classifier.add(Dense(6 ,   activation='relu',  kernel_initializer='random_uniform'))
classifier.add(Dropout(rate=0.1))
# add the output layer
classifier.add(Dense(1 ,   activation='sigmoid',  kernel_initializer='random_uniform'))

# compile the ANN
classifier.compile(optimizer = 'adam',  loss ='binary_crossentropy', metrics = ['accuracy'] )
classifier.fit(X_train, y_train, batch_size = 10, epochs =7)

Epoch 1/7
8000/8000 [==============================] - 6s 790us/step - loss: 0.4931 - acc: 0.7955
Epoch 2/7
8000/8000 [==============================] - 4s 554us/step - loss: 0.4350 - acc: 0.7960 
Epoch 3/7
8000/8000 [==============================] - 4s 500us/step - loss: 0.4317 - acc: 0.7960
Epoch 4/7
8000/8000 [==============================] - 5s 579us/step - loss: 0.4297 - acc: 0.7960
Epoch 5/7
8000/8000 [==============================] - 5s 637us/step - loss: 0.4280 - acc: 0.7999
Epoch 6/7
8000/8000 [==============================] - 5s 630us/step - loss: 0.4265 - acc: 0.8204
Epoch 7/7
8000/8000 [==============================] - 5s 584us/step - loss: 0.4268 - acc: 0.8250


In [28]:

# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(6 ,   activation='relu',  kernel_initializer='random_uniform', input_shape = (X_train.shape[1],)))
    classifier.add(Dense(6 ,   activation='relu',  kernel_initializer='random_uniform'))
    classifier.add(Dense(1 ,   activation='sigmoid',  kernel_initializer='random_uniform'))
    classifier.compile(optimizer = optimizer,  loss ='binary_crossentropy', metrics = ['accuracy'] )
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)



parameters = {'batch_size': [10, 20],
              'epochs': [5, 7],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/5
6400/6400 [==============================] - 2s 346us/step - loss: 0.5013 - acc: 0.7980
Epoch 2/5
6400/6400 [==============================] - 1s 181us/step - loss: 0.4281 - acc: 0.7980
Epoch 3/5
6400/6400 [==============================] - 3s 453us/step - loss: 0.4238 - acc: 0.7980
Epoch 4/5
6400/6400 [==============================] - 3s 500us/step - loss: 0.4201 - acc: 0.7997 0s - loss: 0.4163
Epoch 5/5
6400/6400 [==============================] - 4s 653us/step - loss: 0.4174 - acc: 0.8216
Epoch 1/5
6400/6400 [==============================] - 7s 1ms/step - loss: 0.5053 - acc: 0.7962
Epoch 2/5
6400/6400 [==============================] - 4s 637us/step - loss: 0.4281 - acc: 0.7972
Epoch 3/5
6400/6400 [==============================] - 5s 833us/step - loss: 0.4226 - acc: 0.7972
Epoch 4/5
6400/6400 [==============================] - 5s 749us/step - loss: 0.4174 - acc: 0.8161
Epoch 5/5
6400/6400 [==============================] - 5s 771us/step - loss: 0.4117 - acc: 0.8306
Epoc

6400/6400 [==============================] - 1s 180us/step - loss: 0.4195 - acc: 0.8125
Epoch 6/7
6400/6400 [==============================] - 2s 240us/step - loss: 0.4177 - acc: 0.8245
Epoch 7/7
6400/6400 [==============================] - 3s 500us/step - loss: 0.4158 - acc: 0.8292
Epoch 1/7
6400/6400 [==============================] - ETA: 0s - loss: 0.5203 - acc: 0.797 - 2s 390us/step - loss: 0.5168 - acc: 0.7972
Epoch 2/7
6400/6400 [==============================] - 1s 203us/step - loss: 0.4332 - acc: 0.7980
Epoch 3/7
6400/6400 [==============================] - 1s 185us/step - loss: 0.4276 - acc: 0.7980
Epoch 4/7
6400/6400 [==============================] - 1s 190us/step - loss: 0.4232 - acc: 0.7980
Epoch 5/7
6400/6400 [==============================] - 2s 284us/step - loss: 0.4195 - acc: 0.8119
Epoch 6/7
6400/6400 [==============================] - 1s 181us/step - loss: 0.4171 - acc: 0.8216
Epoch 7/7
6400/6400 [==============================] - 1s 178us/step - loss: 0.4161 - acc:

6400/6400 [==============================] - 2s 383us/step - loss: 0.4271 - acc: 0.7973
Epoch 5/5
6400/6400 [==============================] - 2s 327us/step - loss: 0.4236 - acc: 0.7973
Epoch 1/5
6400/6400 [==============================] - 6s 919us/step - loss: 0.5938 - acc: 0.7950
Epoch 2/5
6400/6400 [==============================] - 2s 278us/step - loss: 0.4451 - acc: 0.7958
Epoch 3/5
6400/6400 [==============================] - 2s 269us/step - loss: 0.4322 - acc: 0.7958
Epoch 4/5
6400/6400 [==============================] - 2s 361us/step - loss: 0.4281 - acc: 0.7958
Epoch 5/5
6400/6400 [==============================] - 2s 248us/step - loss: 0.4255 - acc: 0.7958
Epoch 1/7
6400/6400 [==============================] - 5s 721us/step - loss: 0.5579 - acc: 0.7975
Epoch 2/7
6400/6400 [==============================] - 2s 243us/step - loss: 0.4292 - acc: 0.7980
Epoch 3/7
6400/6400 [==============================] - 2s 272us/step - loss: 0.4189 - acc: 0.7980
Epoch 4/7
6400/6400 [=========

Epoch 7/7
8000/8000 [==============================] - 3s 390us/step - loss: 0.4135 - acc: 0.8304


In [29]:
best_parameters

{'batch_size': 10, 'epochs': 7, 'optimizer': 'rmsprop'}

In [30]:
best_accuracy

0.82999999999999996